## First Level Analysis for Social Prediction

2024 February

*Yiyu Wang*



In [1]:
import os

import glob
import nibabel as nib
import numpy as np
import pandas as pd
import copy

import nilearn
from nilearn import datasets, plotting, image
from nilearn.image import smooth_img, resample_to_img
from nilearn import plotting
from nilearn.masking import apply_mask
from nilearn.input_data import NiftiMasker
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.second_level import SecondLevelModel
from scipy.stats import norm


import gzip

import seaborn as sns
import matplotlib.pyplot as plt

from os.path import join

/Users/yiyuwang/anaconda3/envs/neuroimaging_env/lib/python3.7/site-packages/nilearn/__init__.py:67: FutureWarning: Python 3.7 support is deprecated and will be removed in release 0.12 of Nilearn. Consider switching to Python 3.9 or 3.10.
  _python_deprecation_warnings()
/Users/yiyuwang/anaconda3/envs/neuroimaging_env/lib/python3.7/site-packages/nilearn/input_data/__init__.py:23: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


In [2]:
print(nilearn.__version__)

0.10.2


In [3]:
# local directory set up
#data_dir =base_dir + 'transformed_data_2mm/'

# directory set up for cluster:
# base_dir = '/scratch/wang.yiyu/SocialAbstraction/'
data_dir = '/Users/yiyuwang/Downloads/transformed_data_2mm/'

logfiles_dir = 'Data/logfiles/'
confounds_dir = 'Data/confounds/'
mask_dir = 'masks/'
figures_dir = 'figures/'

subjects_list = pd.read_csv('Data/included_SocialPred_subjects.csv', header=None)
subjects_list = subjects_list[0].values.tolist()
sample_n = len(subjects_list)
print("subjects in this analysis:")
print(subjects_list)
print(f"**** n = {sample_n} *****" )

vmax = 12
TR = .001
N_TR = 675
TR_Length = 0.8
TR_IN_MS = int(TR_Length/TR)

fwhm = 8

# resample a gray matter mask
gm_mask_img = nib.load(mask_dir + 'gm_mask_icbm152_brain.nii.gz')
confounds_of_interest = ['csf',
                        'white_matter',
                        'trans_x', 
                        'trans_y', 
                        'trans_z',
                        'rot_x',
                        'rot_y',
                        'rot_z','framewise_displacement']

subjects in this analysis:
[152, 179, 154, 158, 173, 153, 159, 174, 162, 145, 143, 181, 144, 169, 146, 167, 161, 182, 147, 166, 160, 185, 170, 176, 151, 157, 171, 177, 150, 156]
**** n = 30 *****


In [4]:
logfile_headers =np.array(['obs_video_name', 'fd_video_name','video_number','trial_condition','run_number','run_condition',
             'obs_video_onset','obs_video_offset','obs_video_duration_method1','obs_video_duration_method2',
             'prediction','prediction_x','prediction_y','prediction_RT','prediction_onset', 
             'fb_video_onset','fb_video_offset','fb_video_duration_method1','fb_video_duration_method2',
             'surprise','surprise_RT','surprise_onset'])

In [5]:
def AddSteadyStateOutliers(columns_of_interest, all_columns):
    new_columns = copy.deepcopy(columns_of_interest)
    for column in all_columns:
        if 'outlier' in column:
            new_columns.append(column) 
    return new_columns



def CreateConfoundMatrix(confound_file_path, 
                         confounds_of_interest, s, run):
    
    
    confounds = pd.read_csv(confound_file_path, sep='\t')
    
    confounds_of_interest = AddSteadyStateOutliers(confounds_of_interest, confounds.columns)
    
    cov = confounds[confounds_of_interest]
    cov.values[np.isnan(cov.values)]=0
    return cov

In [6]:
def get_condition(cn):
    if cn == 1:
        cat = 'Pattern'
    elif cn == 2:
        cat = 'Social'
    else:
        print('no such Condition number!')
    return cat 

def parse_task_lines(lines, headers):
    for (i, line) in enumerate(lines):
        cols = line.split(' ')

        video_number = cols[int(np.where(headers == 'video_number')[0])]
        
        trial_condition = int(cols[int(np.where(headers == 'trial_condition')[0])])
        trial_condition = get_condition(trial_condition)
        
        run_condition = int(cols[int(np.where(headers == 'run_condition')[0])])
        run_condition = get_condition(run_condition)

        obs_trial_type = 'obs_' + run_condition

        # congruent means trial_condition == run_condition
        if run_condition == trial_condition:
            if trial_condition == 'Pattern':
                fb_trial_type = 'fb_Pattern_Congruent'
            elif trial_condition == 'Social':
                fb_trial_type = 'fb_Social_Congruent'
            
        else: # PE means trial_condition != run_condition
            if trial_condition == 'Pattern':
                fb_trial_type = 'fb_Social_PE'
            elif trial_condition == 'Social':
                fb_trial_type = 'fb_Pattern_PE'
        
        video_onset = float(cols[int(np.where(headers == 'obs_video_onset')[0])])
        video_offset = float(cols[int(np.where(headers == 'obs_video_offset')[0])])  
        video_duration = video_offset - video_onset
        
        fb_video_onset = float(cols[int(np.where(headers == 'fb_video_onset')[0])])
        fb_video_offset = float(cols[int(np.where(headers == 'fb_video_offset')[0])])  
        fb_video_duration = fb_video_offset - fb_video_onset

        run = int(cols[int(np.where(headers == 'run_number')[0])])
        
        yield [video_onset, video_duration, obs_trial_type, run]
        yield [fb_video_onset,fb_video_duration, fb_trial_type, run]
        # yield [surprise_onset,surprise_duration, 'surprise_onset', run]
        # yield [prediction_onset, prediction_duration, 'pred_onset', run]

            

def create_events_dataframe(task_csv, run):   
    task_lines =[]       
    # df = pd.DataFrame(columns=['onset','duration','trial_type'])
    with open(task_csv, 'r') as task_csv_file:
        task_lines.append(list(parse_task_lines(task_csv_file.readlines()[0:], logfile_headers)))

    df = pd.DataFrame(task_lines[0], columns=['onset','duration','trial_type','run'])
    df= df[df['run']==run].drop(columns=['run'])
    return df

In [14]:

res_dir = f'fmri_results/1stLvl/'
if not os.path.isdir(res_dir):
    os.makedirs(res_dir)

for s in subjects_list:
    print(f'running subject {s}')
    sub_output_dir = res_dir + f'/{s}/'
    if not os.path.isdir(sub_output_dir):
        os.makedirs(sub_output_dir)
        
    task_file = glob.glob(logfiles_dir + f"/*{s}*edited.txt")
    task_csv = task_file[0]
    
    for run in [1,2]:
        events = create_events_dataframe(task_csv, run)

        #get confounds info:
        confounds_str = f'sub-{s}_task-socialpred_run-{run}_desc-confounds_timeseries.tsv'
        cov = CreateConfoundMatrix(confounds_dir + confounds_str, confounds_of_interest, s, run)
        
        fmri_glm = FirstLevelModel(t_r=TR_Length,
                           noise_model='ar3',
                           standardize=True,
                           hrf_model='spm',
                           drift_model='cosine',
                           high_pass=.012, mask_img=gm_mask_img,smoothing_fwhm=8)
        
        func_str = f'sub-{s}_socialpred_run{run}.nii.gz'
        func_path = data_dir + func_str
        fmri_img = nib.load(func_path)
        fmri_glm = fmri_glm.fit(fmri_img, events, confounds=cov)
        
        # save design_matrix for every run
        design_matrix = fmri_glm.design_matrices_[0]

        plotting.plot_design_matrix(design_matrix, output_file=join(sub_output_dir, f'design_matrix_run{run}.png'))
        contrast_matrix = np.eye(design_matrix.shape[1])
        # # extract the betas
        for i in range(3):
            print(f'saving regressor for video {design_matrix.columns[i]}')
            eff = fmri_glm.compute_contrast(contrast_matrix[i],output_type='effect_size')
            nii_file_path = sub_output_dir + f'/sub-{s}_run-{run}_beta_video-{design_matrix.columns[i]}_gm_masked.nii.gz'
            nib.save(eff, nii_file_path)

            eff = fmri_glm.compute_contrast(contrast_matrix[i],output_type='z_score')
            nii_file_path = sub_output_dir + f'/sub-{s}_run-{run}_z_score_video-{design_matrix.columns[i]}_gm_masked.nii.gz'
            nib.save(eff, nii_file_path)

        

running subject 152
saving regressor for video fb_Social_Congruent
saving regressor for video fb_Social_PE
saving regressor for video obs_Social
saving regressor for video fb_Pattern_Congruent
saving regressor for video fb_Pattern_PE
saving regressor for video obs_Pattern
running subject 179
saving regressor for video fb_Pattern_Congruent
saving regressor for video fb_Pattern_PE
saving regressor for video obs_Pattern
saving regressor for video fb_Social_Congruent
saving regressor for video fb_Social_PE
saving regressor for video obs_Social
running subject 154
saving regressor for video fb_Social_Congruent
saving regressor for video fb_Social_PE
saving regressor for video obs_Social
saving regressor for video fb_Pattern_Congruent
saving regressor for video fb_Pattern_PE
saving regressor for video obs_Pattern
running subject 158
saving regressor for video fb_Social_Congruent
saving regressor for video fb_Social_PE
saving regressor for video obs_Social
saving regressor for video fb_Patter